<a href="https://colab.research.google.com/github/srinithish/product-recognition/blob/master/LocalisationAndClassification_Loss_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import sklearn
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import random
import matplotlib.cm as cm
import math
# import FinalPredictionsProcessing
import pickle

In [2]:
# a = tf.ones([3,2,2],dtype=tf.dtypes.float32)
# b = tf.zeros([3,2,2],dtype=tf.dtypes.float32)

grows = 3
gcols = 3
col_id = tf.to_float(tf.reshape(tf.tile(tf.range(grows), [gcols]), (1, grows, gcols, 1)))
row_id = tf.to_float(tf.reshape(tf.tile(tf.range(grows), [gcols]), (1, grows, gcols, 1)))
row_id = tf.transpose(row_id)
coord_id = tf.concat([col_id,row_id],-1)

c = tf.multiply(coord_id,[10,10])

d = tf.reverse(c,[-1])
sess  = tf.InteractiveSession()

Instructions for updating:
Use tf.cast instead.


In [0]:
someTensor = tf.constant([[[1,2],[3,4]]])


# tf.reduce

tf.reduce_sum(someTensor,axis = 2).eval()


array([[3, 7]], dtype=int32)

In [0]:
# https://medium.com/@jonathan_hui/real-time-object-detection-with-yolo-yolov2-28b1b93e2088
# https://github.com/thtrieu/darkflow/blob/master/darkflow/net/yolo/train.py
# https://hackernoon.com/understanding-yolo-f5a74bbc7967

def loss(y_true,y_pred):
    '''
    y_true.shape => (batch,grows,gcols,params)    \\ params = P(obj) + 4 b.b params + each class probabilities
    y_pred.shape => y_true.shape
        
    b.b params => (x,y,h,w)
    
    There are 3 types of losses:
    1. classification loss
    2. localization loss
    3. confidence loss
    
    The λ parameters that appear here and also in the first part are used to 
    differently weight parts of the loss functions. This is necessary to increase 
    model stability. The highest penalty is for coordinate predictions (λ coord = 5) 
    and the lowest for confidence predictions when no object is present (λ noobj = 0.5).
    '''
    
    ##### Confidence loss
    '''
    Most boxes do not contain any objects. This causes a class imbalance problem,
    i.e. we train the model to detect background more frequently than detecting 
    objects. To remedy this, we weight this loss down by a factor λnoobj (default: 0.5).
    box_conf_score = P(obj) * (IoU with g.t)
    '''
    P_obj = y_pred[:,:,:,0]    # Slice probs
    
    # Calculate iou for all the predicted boxes
    # calculate intersection
    denorm_coords = Denormalize_coordinates(y_pred[:,:,:,1:3]) # Slice the x,y
    true_area,pred_area,interset = Areas(denorm_coords,y_pred[:,:,:,3:5])
    
    lefttop,rightbot = Corner_coords(denorm_coords,y_pred[:,:,:,3:5]) # Slice the h,w
    
    
    iou = tf.truediv(intersect, true_area + pred_area - intersect)  # intersection/union
    
    confs = tf.math.multiply(P_obj,iou)
    
    

In [0]:
def Denormalize_coordinates(pred_coords):
    '''
    Inputs
    pred_coords.shape => (batch,grows,gcols,2) # normed x and y of the b.b
    
    Output:
    denormalized_coord.shape => (batch,grows,gcols,2) # denormed x and y of the b.b
    
    generates center X and center Y denormalised
    
    
    '''
    img_height = tf.constant(342)
    img_width = tf.constant(342)
    
    grows = tf.constant(19)
    gcols = tf.constant(19)
    
    cell_height = tf.truediv(img_height,grows)
    cell_width = tf.truediv(img_width,gcols)
    
#     grid_cell_topleft["y"] = grid_row_no*cell_height
#     grid_cell_topleft["x"] = grid_col_no*cell_width
        
    col_id = tf.to_float(tf.reshape(tf.tile(tf.range(grows), [gcols]), (1, grows, gcols, 1)))
    row_id = tf.to_float(tf.reshape(tf.tile(tf.range(grows), [gcols]), (1, grows, gcols, 1)))
    row_id = tf.transpose(row_id)
    coord_id = tf.concat([col_id,row_id],-1) # a 4D tensor that gives the coordinates for every box (col,row)

    coords_lefttop = tf.multiply(coord_id,[cell_width,cell_height]) # Gets the leftmost coordinate for every box
    cell_denormalized_coords = tf.multiply(pred_coords,[cell_width,cell_height])
    
    denormalized_coord = tf.add(coords_lefttop ,cell_denormalized_coords)
    
    return denormalized_coord

In [0]:
def get_denorm_width_height(normHeightWidth_params):
    '''
    We have h,w but we want w,h
    '''
    img_height = tf.constant(342)
    img_width = tf.constant(342)
    
    grows = tf.constant(19)
    gcols = tf.constant(19)
    
    box_params = tf.reverse(normHeightWidth_params,[-1]) # Now w,h
    denormHeightWidth_params = tf.multiply(box_params,[img_width,img_height])
    
    
    return denormHeightWidth_params

In [0]:
def Areas(denorm_coords,y_true,box_params):
    lefttop,rightbot = Corner_coords(denorm_coords,box_params) # Slice the h,w
    
    
    
    

In [0]:

### will get [[xmin,ymin],[xmin,ymin]]

def getLocalisationLoss(yPred,yTrue,lambdaVal):
    
    objectMask = yTrue[:,:,:,0]
    
    objectMask2D =  tf.stack([objectMask,objectMask],axis = 3)
    
    ##for cordinates
    pred_denormalized_coord = Denormalize_coordinates(yPred[:,:,:,1:3])
    true_denormalized_coord = Denormalize_coordinates(yTrue[:,:,:,1:3])
    
   
    sq_subtract_center = tf.square(true_denormalized_coord-pred_denormalized_coord)
    
    masked_sq_subtract_center = tf.multiply(sq_subtract_center,objectMask2D)
    masked_sq_subtract_center = tf.reduce_sum(masked_sq_subtract_center,axis = 3) ##collapse depth
    
    flat_center = tf.layers.flatten(masked_sq_subtract_center)
    batchwiseLoss_center = tf.reduce_sum(flat_center,axis = 1)
    meanLoss_center = tf.reduce_mean(batchwiseLoss_center)
    
    
    
    ###for height and width
    pred_denorm_heightWidth = get_denorm_width_height(yPred[:,:,:,3:5])
    true_denorm_heightWidth = get_denorm_width_height(yTrue[:,:,:,3:5])
    
    
    sq_subtract_sqrt = tf.square(tf.sqrt(pred_denorm_heightWidth) - tf.sqrt(true_denorm_heightWidth))
    
    maskedWidthHeight  = tf.multiply(sq_subtract_sqrt,objectMask2D)
    
    flat_widthHeight = tf.layers.flatten(maskedWidthHeight)
    batchwiseLoss_widthHeight = tf.reduce_sum(flat_widthHeight,axis = 1)
    meanLoss_widthHeight = tf.reduce_mean(batchwiseLoss_widthHeight)
    
    
    
    
    
    
    return lambdaVal*(meanLoss_widthHeight+meanLoss_center)

In [0]:
def get_classificationLoss(yPred,yTrue,lambdaVal):
    
    
    objectMask = yTrue[:,:,:,0]
    
    objectMask2D =  tf.stack([objectMask,objectMask],axis = 3)
    
    trueClassProbabilities = yTrue[:,:,:,5:] ##gets all class probabilities
    predClassProbabilities = yTrue[:,:,:,5:] ###
    
    sq_subtract = tf.square(tf.subtract(trueClassProbabilities,predClassProbabilities))
    sq_subtract = tf.reduce_sum(sq_subtract,axis = 3) ##collapse along depth
    
    masked_sq_subtract = tf.multiply(sq_subtract,objectMask)
    
    flat = tf.layers.flatten(masked_sq_subtract)
    batchwiseLoss = tf.reduce_sum(flat,axis = 1)
    meanLoss = tf.reduce_mean(batchwiseLoss)
    
    return lambdaVal*meanLoss    

In [0]:
dirpathForTrainingFiles  = "/content/Drive/My Drive/Deeplearning Project/Reshaped/"



annotationsPath = dirpathForTrainingFiles + 'annotations/'
imagesPath = dirpathForTrainingFiles+ 'images/'

###for image as numpy array X
imagesAsArrayPklPath =  annotationsPath+'resizedAllImgArray.pkl'
###for true Y
targetVariablePklPath = annotationsPath+'resizedAllTargetArray.pkl'

### for visualising

imagefilePattern = imagesPath+'*'
ImgDictsPath_True_Path = annotationsPath+'resizedImageDictsAllFiles.pkl'
ObjLists_True_Path = annotationsPath+'resizedObjectListsAllFiles.pkl'
predictionArrayPath = annotationsPath+'PredictionArray.pkl'


TrainX = np.array(pickle.load(open(imagesAsArrayPklPath, 'rb')))

TrainY = np.array(pickle.load(open(targetVariablePklPath, 'rb')))
predictionsY = np.array(pickle.load(open(predictionArrayPath, 'rb')))
predictionsY = predictionsY.astype(np.float32)
TrainY = TrainY.astype(np.float32)

In [10]:

sess  = tf.InteractiveSession()
predictionsY = tf.convert_to_tensor(predictionsY)
TrainY = tf.convert_to_tensor(TrainY)

loss = get_classificationLoss(predictionsY[0],TrainY,0.5)


Instructions for updating:
Use keras.layers.flatten instead.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [11]:
loss.eval()

0.0

In [36]:
TrainY[0,3,7,:].eval()

array([0., 0., 0., 0., 0., 0.], dtype=float32)

In [37]:
predictionsY[0,3,7,:].eval()

array([0.        , 1.        , 0.39699072, 1.        , 1.        ,
       1.        ], dtype=float32)